<a href="https://csdms.colorado.edu"><img style="float: center; width: 75%" src="https://raw.githubusercontent.com/csdms/ivy/main/media/logo.png"></a>

# The deAlmeida Overland Flow Component 

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>

This notebook illustrates running the deAlmeida overland flow component in an extremely simple-minded way on a real topography, then shows it creating a flood sequence along an inclined surface with an oscillating water surface at one end.

First, import what we'll need:

In [ ]:
import numpy as np
from landlab.components.overland_flow import OverlandFlow
from landlab.io.esri_ascii import read_esri_ascii
from matplotlib.pyplot import figure

Pick the initial and run conditions

In [ ]:
run_time = 100  # duration of run, (s)
h_init = 0.1  # initial thin layer of water (m)
n = 0.01  # roughness coefficient, (s/m^(1/3))
g = 9.8  # gravity (m/s^2)
alpha = 0.7  # time-step factor (nondimensional; from Bates et al., 2010)
u = 0.4  # constant velocity (m/s, de Almeida et al., 2012)

Elapsed time starts at 1 second. This prevents errors when setting our boundary conditions.

In [ ]:
elapsed_time = 1.0

Use Landlab methods to import an ARC ascii grid, and load the data into the field that the component needs to look at to get the data. This loads the elevation data, z, into a "field" in the grid itself, defined on the nodes.

In [ ]:
rmg, z = read_esri_ascii("data/square-test-basin.asc", name="topographic__elevation")
rmg.set_closed_boundaries_at_grid_edges(True, True, True, True)

# un-comment these two lines for a "real" DEM
# rmg, z = read_esri_ascii('data/hugo_site.asc', name='topographic__elevation')
# rmg.status_at_node[z<0.0] = rmg.BC_NODE_IS_CLOSED

We can get at this data with this syntax:

In [ ]:
np.all(rmg.at_node["topographic__elevation"] == z)

Note that the boundary conditions for this grid mainly got handled with the final line of those three, but for the sake of completeness, we should probably manually "open" the outlet. We can find and set the outlet like this:

In [ ]:
# This DEM was generated using Landlab and the outlet node ID was known
my_outlet_node = 100
rmg.status_at_node[my_outlet_node] = rmg.BC_NODE_IS_FIXED_VALUE

Now initialize a couple more grid fields that the component is going to need:

In [ ]:
rmg.add_full("surface_water__depth", h_init, at="node")  # water depth (m)

Let's look at our watershed topography

In [ ]:
rmg.imshow("topographic__elevation")  # , vmin=1650.0)

Now instantiate the component itself

In [ ]:
# for stability in steeper environments, we set the steep_slopes flag to True
of = OverlandFlow(rmg, steep_slopes=True)

Now we're going to run the loop that drives the component:

In [ ]:
?of.overland_flow

In [ ]:
from tqdm import tqdm

elapsed_time = 1.0
with tqdm(total=run_time) as pbar:
    while elapsed_time < run_time:
        # First, we calculate our time step.
        dt = of.calc_time_step()
        if elapsed_time + dt > run_time:
            dt = run_time - elapsed_time

        # Now, we can generate overland flow.
        of.overland_flow(dt=dt)

        elapsed_time += dt
        pbar.update(dt)

In [ ]:
rmg.imshow("surface_water__depth", cmap="Blues")

Now let's get clever, and run a set of time slices:

In [ ]:
from itertools import pairwise

for start, stop in pairwise([0, 10, 50, 100]):
    of.overland_flow(dt=stop - start)

    figure()
    rmg.imshow("surface_water__depth", cmap="Blues")

### Click here for more <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>